# Explore here

In [1]:
# Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
main_df = pd.read_csv('../data/raw/ems_2025.csv')


main_df

,cad_incident_id,incident_datetime,initial_call_type,initial_severity_level_code,final_call_type,final_severity_level_code,first_assignment_datetime,valid_dispatch_rspns_time_indc,dispatch_response_seconds_qy,first_activation_datetime,...,communitydistrict,communityschooldistrict,congressionaldistrict,reopen_indicator,special_event_indicator,standby_indicator,transfer_indicator,incident_response_seconds_qy,first_to_hosp_datetime,first_hosp_arrival_datetime
0,250010001,2025-01-01 00:00:12,STNDBY,8,STNDBY,8,2025-01-01T09:53:41.000,N,0,2025-01-01T09:57:09.000,...,313.0,21.0,8.0,Y,N,Y,N,NaN,NaN,NaN
1,250010003,2025-01-01 00:01:53,UNC,2,UNC,2,2025-01-01T00:02:06.000,Y,13,2025-01-01T00:02:43.000,...,104.0,2.0,10.0,N,N,N,N,454.0,2025-01-01T00:21:01.000,2025-01-01T00:39:03.000
2,250010004,2025-01-01 00:01:58,CARD,3,CARD,3,2025-01-01T00:01:58.000,Y,0,2025-01-01T00:01:58.000,...,104.0,2.0,10.0,N,N,N,N,0.0,NaN,NaN
3,250010007,2025-01-01 00:03:42,ABDPN,5,ABDPN,5,2025-01-01T00:03:59.000,Y,17,2025-01-01T00:04:12.000,...,112.0,6.0,13.0,N,N,N,N,654.0,2025-01-01T00:28:30.000,2025-01-01T00:37:31.000
4,250010008,2025-01-01 00:04:36,STATEP,2,STATEP,2,2025-01-01T00:05:06.000,Y,30,2025-01-01T00:05:18.000,...,212.0,11.0,16.0,N,N,N,N,284.0,2025-01-01T00:40:27.000,2025-01-01T00:50:50.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079486,252435374,2025-08-31 23:58:45,UNKNOW,4,UNKNOW,4,2025-09-01T00:41:26.000,Y,2561,2025-09-01T00:42:18.000,...,202.0,8.0,15.0,N,N,N,N,NaN,NaN,NaN
1079487,252435375,2025-08-31 23:58:59,OTHER,6,OTHER,6,2025-08-31T23:59:08.000,Y,9,2025-08-31T23:59:21.000,...,414.0,27.0,5.0,N,N,N,N,283.0,NaN,NaN
1079488,252435378,2025-08-31 23:59:22,INJURY,5,INJURY,5,2025-08-31T23:59:37.000,Y,15,2025-08-31T23:59:57.000,...,318.0,22.0,8.0,N,N,N,N,NaN,NaN,NaN
1079489,252435381,2025-08-31 23:59:52,MVAINJ,5,MVAINJ,5,2025-08-31T23:59:52.000,Y,0,2025-08-31T23:59:52.000,...,301.0,14.0,7.0,N,N,N,N,0.0,NaN,NaN


In [ ]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

base_url = "https://data.cityofnewyork.us/resource/76xm-jjuj.json"

#(months 9, 10, 11, 12)
where = "incident_datetime between '2024-09-01T00:00:00' and '2024-12-31T23:59:59'"
LIMIT = 1000
MAX_WORKERS = 12
session = requests.Session()
def fetch_page(offset):
    params = {"$where": where, "$limit": LIMIT, "$offset": offset}
    r = session.get(base_url, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()
    return offset, pd.DataFrame(data)
count_url = base_url
count_params = {"$where": where, "$select": "count(*) as c"}
total = int(session.get(count_url, params=count_params, timeout=30).json()[0]["c"])
print(f"Total records (Sep-Dec 2024): {total:,}")
offsets = list(range(0, total, LIMIT))
dfs = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futures = [ex.submit(fetch_page, off) for off in offsets]
    for fut in as_completed(futures):
        off, df = fut.result()
        print(f"Fetched: {df.shape} | offset: {off}")
        dfs.append(df)
df_fall_2024 = pd.concat(dfs, ignore_index=True)

Total records (Sep-Dec 2024): 537,393
Fetched: (1000, 31) | offset: 0
Fetched: (1000, 31) | offset: 6000
Fetched: (1000, 31) | offset: 7000
Fetched: (1000, 31) | offset: 10000
Fetched: (1000, 31) | offset: 5000
Fetched: (1000, 31) | offset: 9000
Fetched: (1000, 31) | offset: 1000
Fetched: (1000, 31) | offset: 4000
Fetched: (1000, 31) | offset: 8000
Fetched: (1000, 31) | offset: 3000
Fetched: (1000, 31) | offset: 11000
Fetched: (1000, 31) | offset: 16000
Fetched: (1000, 31) | offset: 14000
Fetched: (1000, 31) | offset: 21000
Fetched: (1000, 31) | offset: 13000
Fetched: (1000, 31) | offset: 17000
Fetched: (1000, 31) | offset: 19000
Fetched: (1000, 31) | offset: 15000
Fetched: (1000, 31) | offset: 24000
Fetched: (1000, 31) | offset: 23000
Fetched: (1000, 31) | offset: 22000
Fetched: (1000, 31) | offset: 20000
Fetched: (1000, 31) | offset: 18000
Fetched: (1000, 31) | offset: 26000
Fetched: (1000, 31) | offset: 28000
Fetched: (1000, 31) | offset: 25000
Fetched: (1000, 31) | offset: 29000
Fe

combining data sets

In [5]:
main_df = pd.concat([main_df, df_fall_2024], ignore_index=True)

In [6]:
main_df['initial_call_type'][600]
#CDBRFC is a dispatch/call-type code in the NYC EMS dataset.

'EDPM'

In [7]:
print(f"\n2025 columns: {len(main_df.columns)}")
print(f"2024 columns: {len(df_fall_2024.columns)}")


2025 columns: 31
2024 columns: 31


In [8]:
# Remove leading/trailing spaces from column names
main_df.columns = main_df.columns.str.strip()


In [9]:
main_df["incident_datetime"] = pd.to_datetime(main_df["incident_datetime"], errors="coerce", infer_datetime_format=True)

C:\Users\ayman\AppData\Local\Temp\ipykernel_1992\3193172596.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  main_df["incident_datetime"] = pd.to_datetime(main_df["incident_datetime"], errors="coerce", infer_datetime_format=True)


In [10]:
main_df.columns

Index(['cad_incident_id', 'incident_datetime', 'initial_call_type',
       'initial_severity_level_code', 'final_call_type',
       'final_severity_level_code', 'first_assignment_datetime',
       'valid_dispatch_rspns_time_indc', 'dispatch_response_seconds_qy',
       'first_activation_datetime', 'first_on_scene_datetime',
       'valid_incident_rspns_time_indc', 'incident_travel_tm_seconds_qy',
       'incident_close_datetime', 'held_indicator',
       'incident_disposition_code', 'borough', 'incident_dispatch_area',
       'zipcode', 'policeprecinct', 'citycouncildistrict', 'communitydistrict',
       'communityschooldistrict', 'congressionaldistrict', 'reopen_indicator',
       'special_event_indicator', 'standby_indicator', 'transfer_indicator',
       'incident_response_seconds_qy', 'first_to_hosp_datetime',
       'first_hosp_arrival_datetime'],
      dtype='object')

cleaning up the time 

In [11]:
main_df = main_df.drop_duplicates(subset=['cad_incident_id'])
main_df["incident_datetime"] = pd.to_datetime(main_df["incident_datetime"], errors="coerce",infer_datetime_format=True)
main_df['year'] = main_df['incident_datetime'].dt.year
main_df['month'] =main_df['incident_datetime'].dt.month
main_df['day'] = main_df['incident_datetime'].dt.day
main_df['week_day'] = main_df['incident_datetime'].dt.day_name()
main_df['hour'] = main_df['incident_datetime'].dt.hour
main_df

C:\Users\ayman\AppData\Local\Temp\ipykernel_1992\2757482139.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  main_df["incident_datetime"] = pd.to_datetime(main_df["incident_datetime"], errors="coerce",infer_datetime_format=True)


,cad_incident_id,incident_datetime,initial_call_type,initial_severity_level_code,final_call_type,final_severity_level_code,first_assignment_datetime,valid_dispatch_rspns_time_indc,dispatch_response_seconds_qy,first_activation_datetime,...,standby_indicator,transfer_indicator,incident_response_seconds_qy,first_to_hosp_datetime,first_hosp_arrival_datetime,year,month,day,week_day,hour
0,250010001,2025-01-01 00:00:12,STNDBY,8,STNDBY,8,2025-01-01T09:53:41.000,N,0,2025-01-01T09:57:09.000,...,Y,N,NaN,NaN,NaN,2025.0,1.0,1.0,Wednesday,0.0
1,250010003,2025-01-01 00:01:53,UNC,2,UNC,2,2025-01-01T00:02:06.000,Y,13,2025-01-01T00:02:43.000,...,N,N,454.0,2025-01-01T00:21:01.000,2025-01-01T00:39:03.000,2025.0,1.0,1.0,Wednesday,0.0
2,250010004,2025-01-01 00:01:58,CARD,3,CARD,3,2025-01-01T00:01:58.000,Y,0,2025-01-01T00:01:58.000,...,N,N,0.0,NaN,NaN,2025.0,1.0,1.0,Wednesday,0.0
3,250010007,2025-01-01 00:03:42,ABDPN,5,ABDPN,5,2025-01-01T00:03:59.000,Y,17,2025-01-01T00:04:12.000,...,N,N,654.0,2025-01-01T00:28:30.000,2025-01-01T00:37:31.000,2025.0,1.0,1.0,Wednesday,0.0
4,250010008,2025-01-01 00:04:36,STATEP,2,STATEP,2,2025-01-01T00:05:06.000,Y,30,2025-01-01T00:05:18.000,...,N,N,284.0,2025-01-01T00:40:27.000,2025-01-01T00:50:50.000,2025.0,1.0,1.0,Wednesday,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616879,243664774,NaT,SICK,6,SICK,6,2024-12-31T21:38:16.000,Y,88,2024-12-31T21:41:13.000,...,N,N,456,2024-12-31T22:09:09.000,2024-12-31T22:18:51.000,NaN,NaN,NaN,NaN,NaN
1616880,243664776,NaT,RESPIR,4,RESPIR,4,2024-12-31T21:37:01.000,Y,4,2024-12-31T21:37:28.000,...,N,N,619,2024-12-31T21:59:47.000,2024-12-31T22:06:07.000,NaN,NaN,NaN,NaN,NaN
1616881,243664777,NaT,SICK,6,SICK,6,2024-12-31T21:37:58.000,Y,45,2024-12-31T21:38:19.000,...,N,N,1606,2024-12-31T22:28:03.000,2024-12-31T22:44:48.000,NaN,NaN,NaN,NaN,NaN
1616882,243664778,NaT,SICK,6,SICK,6,2024-12-31T21:37:18.000,Y,4,2024-12-31T21:37:44.000,...,N,N,753,2024-12-31T22:04:47.000,2024-12-31T22:19:08.000,NaN,NaN,NaN,NaN,NaN


Renaming columns more clearly

In [12]:
main_df.columns

Index(['cad_incident_id', 'incident_datetime', 'initial_call_type',
       'initial_severity_level_code', 'final_call_type',
       'final_severity_level_code', 'first_assignment_datetime',
       'valid_dispatch_rspns_time_indc', 'dispatch_response_seconds_qy',
       'first_activation_datetime', 'first_on_scene_datetime',
       'valid_incident_rspns_time_indc', 'incident_travel_tm_seconds_qy',
       'incident_close_datetime', 'held_indicator',
       'incident_disposition_code', 'borough', 'incident_dispatch_area',
       'zipcode', 'policeprecinct', 'citycouncildistrict', 'communitydistrict',
       'communityschooldistrict', 'congressionaldistrict', 'reopen_indicator',
       'special_event_indicator', 'standby_indicator', 'transfer_indicator',
       'incident_response_seconds_qy', 'first_to_hosp_datetime',
       'first_hosp_arrival_datetime', 'year', 'month', 'day', 'week_day',
       'hour'],
      dtype='object')

In [13]:
main_df.rename(columns={
    'cad_incident_id': 'incident_id',
    'incident_datetime': 'datetime',
    'initial_call_type': 'initial_type',
    'initial_severity_level_code': 'initial_severity',
    'final_call_type': 'final_type',
    'final_severity_level_code': 'final_severity',
    'first_assignment_datetime': 'assignment_time',
    'valid_dispatch_rspns_time_indc': 'valid_dispatch_response',
    'dispatch_response_seconds_qy': 'dispatch_response_sec',
    'first_activation_datetime': 'activation_time',
    'first_on_scene_datetime': 'on_scene_time',
    'valid_incident_rspns_time_indc': 'valid_incident_response',
    'incident_response_seconds_qy': 'incident_response_sec',
    'incident_travel_tm_seconds_qy': 'travel_time_sec',
    'first_to_hosp_datetime': 'to_hospital_time',
    'first_hosp_arrival_datetime': 'hospital_arrival_time',
    'incident_close_datetime': 'close_time',
    'held_indicator': 'held',
    'incident_disposition_code': 'disposition',
    'incident_dispatch_area': 'dispatch_area',
    'policeprecinct': 'precinct',
    'citycouncildistrict': 'council_district',
    'communitydistrict': 'community_district',
    'communityschooldistrict': 'school_district',
    'congressionaldistrict': 'congressional_district',
    'reopen_indicator': 'reopened',
    'special_event_indicator': 'special_events',
    'standby_indicator': 'standby',
    'transfer_indicator': 'transferred'}, inplace=True)

main_df

,incident_id,datetime,initial_type,initial_severity,final_type,final_severity,assignment_time,valid_dispatch_response,dispatch_response_sec,activation_time,...,standby,transferred,incident_response_sec,to_hospital_time,hospital_arrival_time,year,month,day,week_day,hour
0,250010001,2025-01-01 00:00:12,STNDBY,8,STNDBY,8,2025-01-01T09:53:41.000,N,0,2025-01-01T09:57:09.000,...,Y,N,NaN,NaN,NaN,2025.0,1.0,1.0,Wednesday,0.0
1,250010003,2025-01-01 00:01:53,UNC,2,UNC,2,2025-01-01T00:02:06.000,Y,13,2025-01-01T00:02:43.000,...,N,N,454.0,2025-01-01T00:21:01.000,2025-01-01T00:39:03.000,2025.0,1.0,1.0,Wednesday,0.0
2,250010004,2025-01-01 00:01:58,CARD,3,CARD,3,2025-01-01T00:01:58.000,Y,0,2025-01-01T00:01:58.000,...,N,N,0.0,NaN,NaN,2025.0,1.0,1.0,Wednesday,0.0
3,250010007,2025-01-01 00:03:42,ABDPN,5,ABDPN,5,2025-01-01T00:03:59.000,Y,17,2025-01-01T00:04:12.000,...,N,N,654.0,2025-01-01T00:28:30.000,2025-01-01T00:37:31.000,2025.0,1.0,1.0,Wednesday,0.0
4,250010008,2025-01-01 00:04:36,STATEP,2,STATEP,2,2025-01-01T00:05:06.000,Y,30,2025-01-01T00:05:18.000,...,N,N,284.0,2025-01-01T00:40:27.000,2025-01-01T00:50:50.000,2025.0,1.0,1.0,Wednesday,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616879,243664774,NaT,SICK,6,SICK,6,2024-12-31T21:38:16.000,Y,88,2024-12-31T21:41:13.000,...,N,N,456,2024-12-31T22:09:09.000,2024-12-31T22:18:51.000,NaN,NaN,NaN,NaN,NaN
1616880,243664776,NaT,RESPIR,4,RESPIR,4,2024-12-31T21:37:01.000,Y,4,2024-12-31T21:37:28.000,...,N,N,619,2024-12-31T21:59:47.000,2024-12-31T22:06:07.000,NaN,NaN,NaN,NaN,NaN
1616881,243664777,NaT,SICK,6,SICK,6,2024-12-31T21:37:58.000,Y,45,2024-12-31T21:38:19.000,...,N,N,1606,2024-12-31T22:28:03.000,2024-12-31T22:44:48.000,NaN,NaN,NaN,NaN,NaN
1616882,243664778,NaT,SICK,6,SICK,6,2024-12-31T21:37:18.000,Y,4,2024-12-31T21:37:44.000,...,N,N,753,2024-12-31T22:04:47.000,2024-12-31T22:19:08.000,NaN,NaN,NaN,NaN,NaN


                                              Fixing negative response time 

In [14]:
main_df.columns

Index(['incident_id', 'datetime', 'initial_type', 'initial_severity',
       'final_type', 'final_severity', 'assignment_time',
       'valid_dispatch_response', 'dispatch_response_sec', 'activation_time',
       'on_scene_time', 'valid_incident_response', 'travel_time_sec',
       'close_time', 'held', 'disposition', 'borough', 'dispatch_area',
       'zipcode', 'precinct', 'council_district', 'community_district',
       'school_district', 'congressional_district', 'reopened',
       'special_events', 'standby', 'transferred', 'incident_response_sec',
       'to_hospital_time', 'hospital_arrival_time', 'year', 'month', 'day',
       'week_day', 'hour'],
      dtype='object')

In [17]:
time_cols = ['dispatch_response_sec', 'incident_response_sec', 'travel_time_sec']

for col in time_cols:
    if col in main_df.columns:
        # Convert to numeric first (handles strings)
        main_df[col] = pd.to_numeric(main_df[col], errors='coerce')
        
        negative_count = (main_df[col] < 0).sum()
        total_count = len(main_df)
        print(f"\n{col}")
        print(f"Total rows: {total_count:,}")
        print(f"Negative values: {negative_count:,}")
        if negative_count > 0:
            negatives = main_df[main_df[col] < 0][col].head()
            print(f"Examples: {negatives.tolist()}")


dispatch_response_sec
Total rows: 1,616,884
Negative values: 0

incident_response_sec
Total rows: 1,616,884
Negative values: 0

travel_time_sec
Total rows: 1,616,884
Negative values: 0


In [18]:
time_cols

['dispatch_response_sec', 'incident_response_sec', 'travel_time_sec']

In [19]:
#time_cols = ['dispatch_response_sec', 'incident_response_sec', 'travel_time_sec'] # 3 time columns (dispatch, incident, travel times)
#for col in time_cols:
    #if col in main_df.columns: #counts how many negative values exist
        #negative_count = (main_df[col] < 0).sum()
        #if negative_count > 0:
            #print(f"correcte{negative_count} negative values in {col}")
            #main_df.loc[main_df[col] < 0, col] = np.nan

In [20]:
print(len(main_df))

1616884


In [21]:
# outliers 
#df_model = main_df[(main_df['response_minutes'] > 0)  &  (main_df['response_minutes'] < 60)].copy() #and does not work 
# Keep only positive times and keeping only times under 60 minutes 

 Clean geographic data 

In [22]:
main_df.columns

Index(['incident_id', 'datetime', 'initial_type', 'initial_severity',
       'final_type', 'final_severity', 'assignment_time',
       'valid_dispatch_response', 'dispatch_response_sec', 'activation_time',
       'on_scene_time', 'valid_incident_response', 'travel_time_sec',
       'close_time', 'held', 'disposition', 'borough', 'dispatch_area',
       'zipcode', 'precinct', 'council_district', 'community_district',
       'school_district', 'congressional_district', 'reopened',
       'special_events', 'standby', 'transferred', 'incident_response_sec',
       'to_hospital_time', 'hospital_arrival_time', 'year', 'month', 'day',
       'week_day', 'hour'],
      dtype='object')

In [23]:
geo_cols = ['zipcode', 'precinct', 'council_district','community_district', 'school_district', 'congressional_district']
#convert to integers

In [24]:
for col in geo_cols:
    if col in main_df.columns:
        main_df[col] = main_df[col].fillna(0).astype(int)

In [25]:
for col in geo_cols:
    if col in main_df.columns:
        main_df[col] = main_df[col].fillna(0).astype(int)
        # main_df[col] = pd.to_numeric(main_df[col], errors="coerce").fillna(0).astype(int)
        print(f"cleaned {col}")

main_df[geo_cols].dtypes

cleaned zipcode
cleaned precinct
cleaned council_district
cleaned community_district
cleaned school_district
cleaned congressional_district


zipcode                   int64
precinct                  int64
council_district          int64
community_district        int64
school_district           int64
congressional_district    int64
dtype: object

In [26]:
main_df[geo_cols].isna().sum()

zipcode                   0
precinct                  0
council_district          0
community_district        0
school_district           0
congressional_district    0
dtype: int64

In [27]:
main_df

,incident_id,datetime,initial_type,initial_severity,final_type,final_severity,assignment_time,valid_dispatch_response,dispatch_response_sec,activation_time,...,standby,transferred,incident_response_sec,to_hospital_time,hospital_arrival_time,year,month,day,week_day,hour
0,250010001,2025-01-01 00:00:12,STNDBY,8,STNDBY,8,2025-01-01T09:53:41.000,N,0,2025-01-01T09:57:09.000,...,Y,N,NaN,NaN,NaN,2025.0,1.0,1.0,Wednesday,0.0
1,250010003,2025-01-01 00:01:53,UNC,2,UNC,2,2025-01-01T00:02:06.000,Y,13,2025-01-01T00:02:43.000,...,N,N,454.0,2025-01-01T00:21:01.000,2025-01-01T00:39:03.000,2025.0,1.0,1.0,Wednesday,0.0
2,250010004,2025-01-01 00:01:58,CARD,3,CARD,3,2025-01-01T00:01:58.000,Y,0,2025-01-01T00:01:58.000,...,N,N,0.0,NaN,NaN,2025.0,1.0,1.0,Wednesday,0.0
3,250010007,2025-01-01 00:03:42,ABDPN,5,ABDPN,5,2025-01-01T00:03:59.000,Y,17,2025-01-01T00:04:12.000,...,N,N,654.0,2025-01-01T00:28:30.000,2025-01-01T00:37:31.000,2025.0,1.0,1.0,Wednesday,0.0
4,250010008,2025-01-01 00:04:36,STATEP,2,STATEP,2,2025-01-01T00:05:06.000,Y,30,2025-01-01T00:05:18.000,...,N,N,284.0,2025-01-01T00:40:27.000,2025-01-01T00:50:50.000,2025.0,1.0,1.0,Wednesday,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616879,243664774,NaT,SICK,6,SICK,6,2024-12-31T21:38:16.000,Y,88,2024-12-31T21:41:13.000,...,N,N,456.0,2024-12-31T22:09:09.000,2024-12-31T22:18:51.000,NaN,NaN,NaN,NaN,NaN
1616880,243664776,NaT,RESPIR,4,RESPIR,4,2024-12-31T21:37:01.000,Y,4,2024-12-31T21:37:28.000,...,N,N,619.0,2024-12-31T21:59:47.000,2024-12-31T22:06:07.000,NaN,NaN,NaN,NaN,NaN
1616881,243664777,NaT,SICK,6,SICK,6,2024-12-31T21:37:58.000,Y,45,2024-12-31T21:38:19.000,...,N,N,1606.0,2024-12-31T22:28:03.000,2024-12-31T22:44:48.000,NaN,NaN,NaN,NaN,NaN
1616882,243664778,NaT,SICK,6,SICK,6,2024-12-31T21:37:18.000,Y,4,2024-12-31T21:37:44.000,...,N,N,753.0,2024-12-31T22:04:47.000,2024-12-31T22:19:08.000,NaN,NaN,NaN,NaN,NaN


Standardize geo names

In [28]:
main_df['borough'] 

0           BROOKLYN
1          MANHATTAN
2          MANHATTAN
3          MANHATTAN
4              BRONX
             ...    
1616879        BRONX
1616880       QUEENS
1616881        BRONX
1616882       QUEENS
1616883       QUEENS
Name: borough, Length: 1616884, dtype: object

In [29]:
main_df['borough'] = main_df['borough'].astype(str).str.strip().str.upper()
print(f"Borough values: {main_df['borough'].unique()}")
main_df['borough'].isna().sum()

Borough values: ['BROOKLYN' 'MANHATTAN' 'BRONX' 'RICHMOND / STATEN ISLAND' 'QUEENS'
 'UNKNOWN']


np.int64(0)

Cleaning the YES and NO 

In [30]:
main_df.columns

Index(['incident_id', 'datetime', 'initial_type', 'initial_severity',
       'final_type', 'final_severity', 'assignment_time',
       'valid_dispatch_response', 'dispatch_response_sec', 'activation_time',
       'on_scene_time', 'valid_incident_response', 'travel_time_sec',
       'close_time', 'held', 'disposition', 'borough', 'dispatch_area',
       'zipcode', 'precinct', 'council_district', 'community_district',
       'school_district', 'congressional_district', 'reopened',
       'special_events', 'standby', 'transferred', 'incident_response_sec',
       'to_hospital_time', 'hospital_arrival_time', 'year', 'month', 'day',
       'week_day', 'hour'],
      dtype='object')

In [31]:
indicator_cols = ['held', 'reopened', 'special_event', 'standby', 'transferred','valid_dispatch_response', 'valid_incident_response']
main_df['reopened']
main_df['held']
main_df['special_events']
#main_df['standby']
#main_df['transferred']
#main_df['valid_dispatch_response']
#main_df['valid_incident_response']
#main_df['close_time']

0          N
1          N
2          N
3          N
4          N
          ..
1616879    N
1616880    N
1616881    N
1616882    N
1616883    N
Name: special_events, Length: 1616884, dtype: object

--- Feature Factors ---

In [32]:
for col in indicator_cols:
    if col in main_df.columns:
        main_df[col] = main_df[col].astype(str).str.strip().str.upper()
        main_df[col] = main_df[col].replace({
            'YES': 'Y', 'NO': 'N', 'TRUE': 'Y', 'FALSE': 'N', '1':'Y', '0':'N', 'NAN': np.nan, 'NONE': np.nan})
        print(f"clean {col}")

clean held
clean reopened
clean standby
clean transferred
clean valid_dispatch_response
clean valid_incident_response


In [33]:
#main_df[indicator_cols].isna().sum()
for col in indicator_cols:
    if col in main_df.columns:
        print(f"{col} - EXISTS")
    else:
        print(f"{col} - NOT FOUND")

held - EXISTS
reopened - EXISTS
special_event - NOT FOUND
standby - EXISTS
transferred - EXISTS
valid_dispatch_response - EXISTS
valid_incident_response - EXISTS


In [34]:
main_df.isna().sum()

incident_id                     0
datetime                   537393
initial_type                    0
initial_severity                0
final_type                      0
final_severity                  0
assignment_time             33254
valid_dispatch_response         0
dispatch_response_sec           0
activation_time             36257
on_scene_time               96128
valid_incident_response         0
travel_time_sec             96155
close_time                    217
held                            0
disposition                     0
borough                         0
dispatch_area                   0
zipcode                         0
precinct                        0
council_district                0
community_district              0
school_district                 0
congressional_district          0
reopened                        0
special_events                  0
standby                         0
transferred                     0
incident_response_sec       96508
to_hospital_ti

In [35]:
print(f"Columns: {len(main_df.columns)}")

Columns: 36


In [36]:
main_df['year'].max()           #2025
main_df['year'].min() # 2025
main_df['month'].max()           #8
main_df['month'].min()   #1
main_df['day'].max()             #31
main_df['day'].min()     #1

#"start date": "2025-01-01",
#"end date": "2025-08-31",

np.float64(1.0)

weekend cleaning

In [37]:
#fixing issue with weekends

main_df['is_weekend'] = main_df['week_day'].isin(['Saturday','Sunday']).astype(int)

#make new column for rush hour time to help with the model. 
main_df['is_rush_hour'] = main_df['hour'].isin([7,8,9,16,17,18,19]).astype(int)
#These hours correspond to morning 7–9 AM and evening 4–7 PM rush hours
#most action around 14 hr   3359 or 2pm 
# 2pm

Adding rush hour time

In [38]:
def time_category(hour):
    if 6 <= hour < 12: return 'morning'
    elif 12 <= hour < 18: return 'afternoon'
    elif 18 <= hour < 22: return 'evening'
    else: return 'night'

main_df['time_of_day'] = main_df['hour'].apply(time_category)

Season engineering

In [39]:
main_df['season'] = main_df['month'].map({
    1: 'winter', 2: 'winter',
    3: 'spring', 4: 'spring', 5: 'spring',
    6: 'summer', 7: 'summer', 8: 'summer',
    9: 'fall', 10: 'fall', 11: 'fall', 12: 'winter'})

In [51]:
from pandas.tseries.holiday import USFederalHolidayCalendar

holiday = USFederalHolidayCalendar()
holidays = holiday.holidays(start='2024-01-01', end='2025-12-31')

holiday_dates = holidays.date


main_df['datetime'] = pd.to_datetime(main_df['datetime'])
main_df['is_holiday'] = main_df['datetime'].dt.normalize().isin(holidays).astype(int)

print(holidays)

DatetimeIndex(['2024-01-01', '2024-01-15', '2024-02-19', '2024-05-27',
               '2024-06-19', '2024-07-04', '2024-09-02', '2024-10-14',
               '2024-11-11', '2024-11-28', '2024-12-25', '2025-01-01',
               '2025-01-20', '2025-02-17', '2025-05-26', '2025-06-19',
               '2025-07-04', '2025-09-01', '2025-10-13', '2025-11-11',
               '2025-11-27', '2025-12-25'],
              dtype='datetime64[ns]', freq=None)


In [52]:
main_df

,incident_id,datetime,initial_type,initial_severity,final_type,final_severity,assignment_time,valid_dispatch_response,dispatch_response_sec,activation_time,...,month,day,week_day,hour,is_weekend,is_rush_hour,time_of_day,season,is_holiday,datetime_hour
0,250010001,2025-01-01 00:00:12,STNDBY,8,STNDBY,8,2025-01-01T09:53:41.000,N,0,2025-01-01T09:57:09.000,...,1.0,1.0,Wednesday,0.0,0,0,night,winter,1,2025-01-01
1,250010003,2025-01-01 00:01:53,UNC,2,UNC,2,2025-01-01T00:02:06.000,Y,13,2025-01-01T00:02:43.000,...,1.0,1.0,Wednesday,0.0,0,0,night,winter,1,2025-01-01
2,250010004,2025-01-01 00:01:58,CARD,3,CARD,3,2025-01-01T00:01:58.000,Y,0,2025-01-01T00:01:58.000,...,1.0,1.0,Wednesday,0.0,0,0,night,winter,1,2025-01-01
3,250010007,2025-01-01 00:03:42,ABDPN,5,ABDPN,5,2025-01-01T00:03:59.000,Y,17,2025-01-01T00:04:12.000,...,1.0,1.0,Wednesday,0.0,0,0,night,winter,1,2025-01-01
4,250010008,2025-01-01 00:04:36,STATEP,2,STATEP,2,2025-01-01T00:05:06.000,Y,30,2025-01-01T00:05:18.000,...,1.0,1.0,Wednesday,0.0,0,0,night,winter,1,2025-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616879,243664774,NaT,SICK,6,SICK,6,2024-12-31T21:38:16.000,Y,88,2024-12-31T21:41:13.000,...,NaN,NaN,NaN,NaN,0,0,night,NaN,0,NaT
1616880,243664776,NaT,RESPIR,4,RESPIR,4,2024-12-31T21:37:01.000,Y,4,2024-12-31T21:37:28.000,...,NaN,NaN,NaN,NaN,0,0,night,NaN,0,NaT
1616881,243664777,NaT,SICK,6,SICK,6,2024-12-31T21:37:58.000,Y,45,2024-12-31T21:38:19.000,...,NaN,NaN,NaN,NaN,0,0,night,NaN,0,NaT
1616882,243664778,NaT,SICK,6,SICK,6,2024-12-31T21:37:18.000,Y,4,2024-12-31T21:37:44.000,...,NaN,NaN,NaN,NaN,0,0,night,NaN,0,NaT


In [53]:
main_df.columns

Index(['incident_id', 'datetime', 'initial_type', 'initial_severity',
       'final_type', 'final_severity', 'assignment_time',
       'valid_dispatch_response', 'dispatch_response_sec', 'activation_time',
       'on_scene_time', 'valid_incident_response', 'travel_time_sec',
       'close_time', 'held', 'disposition', 'borough', 'dispatch_area',
       'zipcode', 'precinct', 'council_district', 'community_district',
       'school_district', 'congressional_district', 'reopened',
       'special_events', 'standby', 'transferred', 'incident_response_sec',
       'to_hospital_time', 'hospital_arrival_time', 'year', 'month', 'day',
       'week_day', 'hour', 'is_weekend', 'is_rush_hour', 'time_of_day',
       'season', 'is_holiday', 'datetime_hour'],
      dtype='object')

Combining data

In [54]:
df = pd.read_csv('../data/raw/nyc_weather_2025.csv')
# so how do i reallt get these two datsets to match ?
# specifically the weather date to the 911 call date
df['datetime'] = pd.to_datetime(df['datetime'])
main_df['datetime_hour'] = main_df['datetime'].dt.floor('H')

merged_df = main_df.merge(df,left_on='datetime_hour',right_on='datetime', how='left',suffixes=('', '_weather'))

print(len(merged_df))
merged_df

C:\Users\ayman\AppData\Local\Temp\ipykernel_1992\2649335757.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  main_df['datetime_hour'] = main_df['datetime'].dt.floor('H')


1616884


,incident_id,datetime,initial_type,initial_severity,final_type,final_severity,assignment_time,valid_dispatch_response,dispatch_response_sec,activation_time,...,is_rush_hour,time_of_day,season,is_holiday,datetime_hour,datetime_weather,temperture,precipitation,windspeed,weathercode
0,250010001,2025-01-01 00:00:12,STNDBY,8,STNDBY,8,2025-01-01T09:53:41.000,N,0,2025-01-01T09:57:09.000,...,0,night,winter,1,2025-01-01,2025-01-01,46.8,0.0,10.7,3.0
1,250010003,2025-01-01 00:01:53,UNC,2,UNC,2,2025-01-01T00:02:06.000,Y,13,2025-01-01T00:02:43.000,...,0,night,winter,1,2025-01-01,2025-01-01,46.8,0.0,10.7,3.0
2,250010004,2025-01-01 00:01:58,CARD,3,CARD,3,2025-01-01T00:01:58.000,Y,0,2025-01-01T00:01:58.000,...,0,night,winter,1,2025-01-01,2025-01-01,46.8,0.0,10.7,3.0
3,250010007,2025-01-01 00:03:42,ABDPN,5,ABDPN,5,2025-01-01T00:03:59.000,Y,17,2025-01-01T00:04:12.000,...,0,night,winter,1,2025-01-01,2025-01-01,46.8,0.0,10.7,3.0
4,250010008,2025-01-01 00:04:36,STATEP,2,STATEP,2,2025-01-01T00:05:06.000,Y,30,2025-01-01T00:05:18.000,...,0,night,winter,1,2025-01-01,2025-01-01,46.8,0.0,10.7,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616879,243664774,NaT,SICK,6,SICK,6,2024-12-31T21:38:16.000,Y,88,2024-12-31T21:41:13.000,...,0,night,NaN,0,NaT,NaT,NaN,NaN,NaN,NaN
1616880,243664776,NaT,RESPIR,4,RESPIR,4,2024-12-31T21:37:01.000,Y,4,2024-12-31T21:37:28.000,...,0,night,NaN,0,NaT,NaT,NaN,NaN,NaN,NaN
1616881,243664777,NaT,SICK,6,SICK,6,2024-12-31T21:37:58.000,Y,45,2024-12-31T21:38:19.000,...,0,night,NaN,0,NaT,NaT,NaN,NaN,NaN,NaN
1616882,243664778,NaT,SICK,6,SICK,6,2024-12-31T21:37:18.000,Y,4,2024-12-31T21:37:44.000,...,0,night,NaN,0,NaT,NaT,NaN,NaN,NaN,NaN


In [55]:
missing_weather = merged_df[['temperture', 'precipitation', 'windspeed', 'weathercode']].isna().sum()
print(f"Missing weather data")


for col, count in missing_weather.items():
    pct = (count / len(merged_df) * 100)
    print(f"  {col}: {count:,} ({pct:.1f}%)")
# good merge

Missing weather data
  temperture: 537,393 (33.2%)
  precipitation: 537,393 (33.2%)
  windspeed: 537,393 (33.2%)
  weathercode: 537,393 (33.2%)


In [56]:
merged_df.columns

Index(['incident_id', 'datetime', 'initial_type', 'initial_severity',
       'final_type', 'final_severity', 'assignment_time',
       'valid_dispatch_response', 'dispatch_response_sec', 'activation_time',
       'on_scene_time', 'valid_incident_response', 'travel_time_sec',
       'close_time', 'held', 'disposition', 'borough', 'dispatch_area',
       'zipcode', 'precinct', 'council_district', 'community_district',
       'school_district', 'congressional_district', 'reopened',
       'special_events', 'standby', 'transferred', 'incident_response_sec',
       'to_hospital_time', 'hospital_arrival_time', 'year', 'month', 'day',
       'week_day', 'hour', 'is_weekend', 'is_rush_hour', 'time_of_day',
       'season', 'is_holiday', 'datetime_hour', 'datetime_weather',
       'temperture', 'precipitation', 'windspeed', 'weathercode'],
      dtype='object')

            Identity & Time:

['incident_id'] =  Unique ID for each 911 call 

['datetime'] = (date/time) When the 911 call came in

['datetime_hour'] = I rounded to the hour for merging with weather

['datetime_weather'] = Duplicate datetime from weather merge (not needed)

    


                Location:

['borough'] = Which NYC borough (Manhattan, Brooklyn, Queens, Bronx, Staten Island)

['dispatch_area'] - Specific dispatch zone within borough

['zipcode'] = ZIP code of incident

['precinct'] = Police precinct number

['council_district'] = City council district

['community_district'] = Community board district

['school_district'] - School district number

['congressional_district'] = US congressional district




            Call Classification:

['initial_type'] = What type of emergency was reported initially (fire, medical, injury, etc.)

['initial_severity'] = How serious it seemed at first (1-10 scale)

['final_type'] = What it actually turned out to be after investigation

['final_severity'] = Actual severity level after assessment





            Timeline of Response:

['assignment_time'] = When a unit was assigned to respond  

['activation_time'] = When the unit was activated/dispatched  

['on_scene_time'] = When responders arrived at the scene   

['to_hospital_time'] - When patient started going to hospital

['hospital_arrival_time'] = When they arrived at hospital

['close_time'] = When the incident was officially closed





            Response Metrics:

['dispatch_response_sec'] = How long from call to dispatch (seconds)

['travel_time_sec'] = How long it took to get to the scene

['incident_response_sec'] = Total response time (Target Variable)






            Quality Flags:

['valid_dispatch_response'] = Is the dispatch time data valid/reliable?

['valid_incident_response'] = Is the response time data valid/reliable?






            Incident Characteristics:

['held'] = Was the call put on hold/delayed before dispatch? (Y/N)


['reopened'] = Did they have to reopen this incident later? (Y/N)


['special_events'] = Was there a special event happening (parade, concert, etc.)? (Y/N)


['standby'] = Was unit on standby? (Y/N)


['transferred'] - Was call transferred to another unit? (Y/N)


['disposition'] - Final outcome code (transported, refused care, cancelled, etc.)





            Time Features (I have created):

['year'], ['month'], ['day'] = Date components']

['hour'] = Hour of day (0-23)

['week_day'] = Day of week (Monday, Tuesday, etc.)





            Weather (from merge):

['temperture'] = Temperature in Fahrenheit (typo in original data)

['precipitation'] = Rain/snow amount

['windspeed'] = Wind speed

['weathercode'] = Weather condition code (0=clear, 71=snow, etc.)



temp merged to work on the location 

In [57]:
merged_df.to_csv('../src/temp_merged_2025_working.csv', index = False)

In [ ]:
# Save the explore notebook
#git add src/explore.ipynb

# Commit it
#git commit -m "Update explore notebook"

# Push to GitHub
#git push origin main